#======Importação dos Dados======#
Nesta etapa, foi realizada a leitura de todos os arquivos CSV da base de dados do Olist, localizados na pasta data/raw/.
Utilizei um dicionário (file_map) para mapear nomes intuitivos aos nomes dos arquivos. Em seguida, carreguei os dados com pandas.read_csv() e armazenamos os DataFrames em um dicionário (dfs) para facilitar o acesso e manipulação.
Por fim, foi exibida as primeiras linhas de cada tabela para verificar se os dados foram carregados corretamente.

In [ ]:
import pandas as pd
import os

#===Definindo o caminho dos arquivos brutos===#
raw_path = '../data/raw/'

#===Mapeando os arquivos em um dicionário para facilitar o carregamento===#
file_map = {
    "orders": "olist_orders_dataset.csv",
    "customers": "olist_customers_dataset.csv",
    "products": "olist_products_dataset.csv",
    "sellers": "olist_sellers_dataset.csv",
    "order_items": "olist_order_items_dataset.csv",
    "order_reviews": "olist_order_reviews_dataset.csv",
    "order_payments": "olist_order_payments_dataset.csv",
    "geolocation": "olist_geolocation_dataset.csv",
    "product_translation": "product_category_name_translation.csv"
}

#===Carregando todos os arquivos em dataframes===#
dfs = {name: pd.read_csv(os.path.join(raw_path, filename)) for name, filename in file_map.items()}

#===Exibindo os primeiros registros de cada dataset===#
for name, df in dfs.items():
    print(f"\n{name.upper()}:\n")
    display(df.head())
